# Goal: Clean and Prepare Global EV Data (Top 4 Markets) for Tableau Dashboard

## Dataset
IEA Global EV Data 2024

## Objective
Prepare a clean, business-focused dataset for Tableau analysis.


In [2]:
import pandas as pd
import numpy as np


In [3]:

file_name = "IEA Global EV Data 2024.csv"  
df = pd.read_csv(file_name)

df.shape
df.columns



Index(['region', 'category', 'parameter', 'mode', 'powertrain', 'year', 'unit',
       'value'],
      dtype='object')

In [4]:
df.head(10)

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000
5,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000
6,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000
7,Australia,Historical,EV stock share,Cars,EV,2012,percent,0.00240
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.00000
9,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.00000


## Step 5.1 – Filter Historical Data

The dataset includes both historical data and future scenarios.
For this project, we only keep historical data to avoid mixing real values with projections.


In [5]:
# Keep only historical data (exclude projections and scenarios)
df_hist = df[df["category"] == "Historical"]

# Check the new dataset size after filtering
df_hist.shape
df_hist.head(10)

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000
5,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000
6,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000
7,Australia,Historical,EV stock share,Cars,EV,2012,percent,0.00240
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.00000
9,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.00000


## Step 5.2 – Filter Vehicle Mode (Cars Only)

The dataset contains different transport modes.
To keep the analysis focused and comparable, we only analyse passenger cars.


In [6]:
# Filter the dataset to include only passenger cars
df_hist = df_hist[df_hist["mode"] == "Cars"]

# Check dataset size after filtering by vehicle mode
df_hist.shape


(3798, 8)

## Standarsisation USA Labels

Region names were standardised to avoid inconsistent labelling across the dataset.
All entries referring to the United States were consolidated under a single category
("United States") to ensure accurate filtering and comparison.


In [7]:
df_hist["region"] = df_hist["region"].replace({
    "United States of America": "United States",
    "USA": "United States"
})


## Step 5.3 – Select Top 4 Markets

To keep the analysis business-focused and readable,
we limit the scope to the four most important EV markets:
World, China, Europe, and United States.


In [8]:
# Define the top 4 EV markets
top_markets = ["World", "China", "Europe", "United States"]

# Filter dataset to keep only the selected markets
df_hist = df_hist[df_hist["region"].isin(top_markets)]

# Verify that only the four selected regions remain
df_hist["region"].unique()



array(['China', 'Europe', 'United States', 'World'], dtype=object)

## Step 5.4 – Select Relevant EV Metrics

The dataset includes many indicators.
For this dashboard, we focus on three key business metrics:
- EV sales
- EV stock
- EV stock share (market share)


In [9]:
# Define the EV metrics relevant for business analysis
metrics = ["EV sales", "EV stock", "EV stock share"]

# Filter dataset to keep only the selected metrics
df_hist = df_hist[df_hist["parameter"].isin(metrics)]

# Check which metrics remain after filtering
df_hist["parameter"].unique()


array(['EV sales', 'EV stock', 'EV stock share'], dtype=object)

## Step 6 – Reshape Data for Tableau

To make the dataset suitable for Tableau dashboards, the data is reshaped from a
long format to a wide format. Each row represents a year and a region, with
separate columns for key EV metrics.


In [10]:
# Pivot the dataset to create one row per year and region
df_pivot = df_hist.pivot_table(
    index=["year", "region"],
    columns="parameter",
    values="value",
    aggfunc="sum"
).reset_index()

df_pivot.head()


parameter,year,region,EV sales,EV stock,EV stock share
0,2010,China,1440.0,1940.0,0.0031
1,2010,Europe,1837.0,9903.0,0.0039
2,2010,United States,1200.0,3800.0,0.0017
3,2010,World,7479.0,20445.0,0.0023
4,2011,China,5120.0,6960.0,0.0092


The column index name created during the pivot operation is removed to avoid
confusion and ensure a clean dataset structure.


In [11]:
# Remove column index name created by the pivot operation
df_pivot.columns.name = None

df_pivot.head()

,year,region,EV sales,EV stock,EV stock share
0,2010,China,1440.0,1940.0,0.0031
1,2010,Europe,1837.0,9903.0,0.0039
2,2010,United States,1200.0,3800.0,0.0017
3,2010,World,7479.0,20445.0,0.0023
4,2011,China,5120.0,6960.0,0.0092


## Step 6.1 – Data Quality Checks

Basic data quality checks are performed to ensure the dataset is complete and
ready for visual analysis.


In [12]:
df_pivot.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            56 non-null     int64  
 1   region          56 non-null     object 
 2   EV sales        56 non-null     float64
 3   EV stock        56 non-null     float64
 4   EV stock share  56 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.3+ KB


In [13]:
df_pivot.isna().sum()


parameter
year              0
region            0
EV sales          0
EV stock          0
EV stock share    0
dtype: int64

In [12]:
df_pivot.to_csv(
    "ev_top4_markets_tableau_ready.csv",
    index=False
)
